In [ ]:
# Dependecies
!pip install transformers
!pip install datasets
!pip install trl

In [ ]:
from datasets import load_dataset

# Загрузка статей с Википедии по теме "формальные языки"
dataset = load_dataset("wikipedia", "20220301.en", split="train")
subset = dataset.filter(lambda x: "formal language" in x['text'] or "regular expressions" in x['text'])

print(subset)

In [ ]:
from transformers import AutoModelForCausalLM
from datasets import load_dataset 
from trl import SFTTrainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
trainer = SFTTrainer(
    model, 
    train_dataset=subset,
    dataset_text_field="text",
    max_seq_length=512,
)
trainer.train()

In [ ]:
from transformers import GPT2Tokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Загрузка токенизатора для DistilGPT-2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")

def ask_question(model, question, max_length=50):
    # Токенизация вопроса
    inputs = tokenizer(question, return_tensors="pt").to(device)
    
    # Генерация ответа моделью
    outputs = model.generate(
        inputs['input_ids'], 
        max_length=max_length, 
        num_beams=5, 
        early_stopping=True
    )
    
    # Декодирование ответа
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

question = "what is regular expression?"

answer = ask_question(model, question)

print("Question:", question)
print("Answer:", answer)
